<a href="https://colab.research.google.com/github/liuyao12/Ranger-Mish-ImageWoof-5/blob/master/Woof_128_ResNet_with_a_Twist.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ResNet with a Twist

> ConvTwist + Ranger + Mish + MaxBlurPool + restrick

# setup and imports

In [ ]:
# pip install git+https://github.com/kornia/kornia

In [3]:
pip install git+https://github.com/ayasyrev/model_constructor

  Cloning https://github.com/ayasyrev/model_constructor to /tmp/pip-req-build-4hlk1_wt
  Running command git clone -q https://github.com/ayasyrev/model_constructor /tmp/pip-req-build-4hlk1_wt
  Created wheel for model-constructor: filename=model_constructor-0.1.1-cp36-none-any.whl size=23583 sha256=0e42aadb46cee427ad6d7976e4d8229f743359fe015f3d7cafb27e521924590d
  Stored in directory: /tmp/pip-ephem-wheel-cache-j4jzx6qe/wheels/5b/92/65/8093a307d5802f41f4a8776b40bc12b558e75a2a906ae8b683
Successfully built model-constructor


In [4]:
pip install git+https://github.com/ayasyrev/imagenette_experiments

  Cloning https://github.com/ayasyrev/imagenette_experiments to /tmp/pip-req-build-pm9fbqhh
  Running command git clone -q https://github.com/ayasyrev/imagenette_experiments /tmp/pip-req-build-pm9fbqhh
     |████████████████████████████████| 163kB 4.5MB/s 
  Created wheel for imagenette-experiments: filename=imagenette_experiments-0.0.1-cp36-none-any.whl size=15332 sha256=5eb01035c9960ecb4685c78d907e47976eb9e19858699729c0972c22a5e9c4d8
  Stored in directory: /tmp/pip-ephem-wheel-cache-2wl3br05/wheels/af/99/98/2212941f45a18bf6d191f56c39e3569138414560c76defc0d4
Successfully built imagenette-experiments
ERROR: kornia 0.3.1 has requirement torch==1.5.0, but you'll have torch 1.5.1+cu101 which is incompatible.


In [5]:
from imagenette_experiments.train_utils import *

In [6]:
from kornia.contrib import MaxBlurPool2d

In [7]:
from fastai.basic_train import *
from fastai.vision import *
# from fastai.script import *
from model_constructor.net import Net, act_fn
from model_constructor.layers import SimpleSelfAttention, ConvLayer

# Twist

In [8]:
class ConvTwist(nn.Module):  # replacing 3x3 Conv2d with in_channels=out_channels
    def __init__(self, ni):
        super(ConvTwist, self).__init__()
        self.twist = True # and ni<=256
        self.iters = 1 #3 if ni<=256 else 1
        self.mask1 = torch.zeros((ni,ni,1,1)) # to mask out the weights of conv1x1
        if self.twist:
            self.expansion = 2
            ni *= self.expansion
            self.conv = nn.Conv2d(ni, ni, kernel_size=3, padding=1, bias=False, groups=1)
            Dx = torch.Tensor([[-1,0,1],[-2,0,2],[-1,0,1]]).view(1,1,3,3)
            Dy = torch.Tensor([[1,2,1],[0,0,0],[-1,-2,-1]]).view(1,1,3,3)
            self.DD = torch.cat([Dx,Dy]*(ni//2), dim=0)
            self.conv1x1 = nn.Conv2d(ni, ni, kernel_size=1, bias=False, groups=1)
            self.mask2 = torch.zeros((ni,ni,1,1)) # to mask out the weights of conv1x1
            random_indices = torch.randint(0,4,(ni,2))
            for i in range(ni//2):
                # self.mask1[i,i] = 1
                self.mask2[i*2:i*2+2,i*2:i*2+2] = 1
                # for k in range(2):
                #     j = min(i//4*4 + random_indices[i,k].item(), ni//2)
                #     # self.mask1[i,j] = 1
                #     self.mask2[i*2:i*2+2,j*2:j*2+2] = 1
            self.XY = None
        else:
            self.conv = nn.Conv2d(ni, ni, kernel_size=3, padding=1, bias=False, groups=1)

    def forward(self, x):
        N,C,H,W = x.size()
        # out = x
        if self.twist:
            CC = C*self.expansion
            out = x.unsqueeze(2).expand(N,C,CC//C,H,W).reshape(N,CC,H,W)
            if self.XY is None:
                XX = torch.from_numpy(np.indices((1,1,H,W))[3]*2/W-1)
                YY = torch.from_numpy(np.indices((1,1,H,W))[2]*2/H-1)
                self.XY = torch.cat([XX,YY]*(CC//2), dim=1).type(x.dtype).to(x.device)
                print(self.XY.size())
                self.DD = self.DD.to(x.device)
                self.mask1 = self.mask1.to(x.device)
                self.mask2 = self.mask2.to(x.device)
            pdf = F.relu(out)+0.01
            total = pdf.sum((2,3))
            XYbar = (self.XY*pdf).sum((2,3))/total
            XY = self.XY-XYbar[:,:,None,None]
            # self.conv1x1.weight.data = self.conv1x1.weight*self.mask2 
            for i in range(self.iters):
                # if i>0: double = out.unsqueeze(2).expand(N,C,CC//C,H,W).reshape(N,CC,H,W)
                twist = F.conv2d(out, self.DD, padding=1, groups=CC)
                twist = self.conv(out) + XY*self.conv1x1(twist)
                # twist = twist.view(N,C,CC//C,H,W).sum(2)
                # twist = twist.view(N,C//4,4,H,W)
                # twist = torch.cat([twist[:,:,1:], twist[:,:,:1]], dim=2).view(N,C,H,W)
                out = out + twist
            out = out.view(N,C,CC//C,H,W).sum(2) - x
        else:
            out = self.conv(out)
        return out

In [ ]:
A = torch.arange(12).view(3,4)
print(A.size())
A.unsqueeze_(1)
print(A.size())
A = A.expand((3,2,4)).reshape(6,4)
print(A)

torch.Size([3, 4])
torch.Size([3, 1, 4])
tensor([[ 0,  1,  2,  3],
        [ 0,  1,  2,  3],
        [ 4,  5,  6,  7],
        [ 4,  5,  6,  7],
        [ 8,  9, 10, 11],
        [ 8,  9, 10, 11]])


# ResBlock

In [9]:
class NewLayer(nn.Sequential):
    """Basic conv layers block"""
    def __init__(self, ni, nf, ks=3, stride=1,
            act=True,  act_fn=nn.ReLU(inplace=True),
            bn_layer=True, bn_1st=True, zero_bn=False,
            padding=None, bias=False, groups=1, **kwargs):

        if padding==None: padding = ks//2
        if ni==nf and ks==3:  layers = [('ConvTwist', ConvTwist(ni))]
        else: layers = [('Conv{}x{}'.format(ks,ks), 
                  nn.Conv2d(ni, nf, ks, stride=stride, padding=padding, bias=bias, groups=groups))]

        act_bn = [('act_fn', act_fn)] if act else []
        if bn_layer:
            bn = nn.BatchNorm2d(nf)
            nn.init.constant_(bn.weight, 0. if zero_bn else 1.)
            act_bn += [('bn', bn)]
        if bn_1st: act_bn.reverse()
        layers += act_bn
        super().__init__(OrderedDict(layers))

In [10]:
class NewResBlock(Module):
    def __init__(self, expansion, ni, nh, stride=1,
                 conv_layer=ConvLayer, act_fn=act_fn, bn_1st=True,
                 pool=nn.AvgPool2d(2, ceil_mode=True), sa=False, sym=False, zero_bn=True):
        nf,ni = nh*expansion,ni*expansion
        conv_layer = NewLayer
        self.reduce = noop if stride==1 else pool
        layers  = [(f"conv_0", conv_layer(ni, nh, 3, act_fn=act_fn, bn_1st=bn_1st)),
                   (f"conv_1", conv_layer(nh, nf, 3, zero_bn=zero_bn, act=False, bn_1st=bn_1st))
        ] if expansion == 1 else [
                   (f"conv_0", conv_layer(ni, nh, 1, act_fn=act_fn, bn_1st=bn_1st)),
                   (f"conv_1", conv_layer(nh, nh, 3, act_fn=act_fn, bn_1st=bn_1st)),
                   (f"conv_2", conv_layer(nh, nf, 1, zero_bn=zero_bn, act=False, bn_1st=bn_1st))
        ]
        if sa: layers.append(('sa', SimpleSelfAttention(nf,ks=1,sym=sym)))
        self.convs = nn.Sequential(OrderedDict(layers))
        self.idconv = noop if ni==nf else conv_layer(ni, nf, 1, act=False, bn_1st=bn_1st)
        self.merge = act_fn

    def forward(self, x):
        o = self.reduce(x)
        return self.merge(self.convs(o) + self.idconv(o))

# Model Constructor

In [11]:
model = Net(c_out=10, layers=[3,6,8,3], expansion=4)
model.block = NewResBlock
# model.conv_layer = NewLayer # for the stem
pool = MaxBlurPool2d(3, True)
model.pool = pool
model.stem_pool = pool
model.stem_sizes = [3,32,64,64]
model.act_fn = Mish()
model.sa = True
res = []

# Runs and results



In [12]:
epochs = [5,5,5]
for e in epochs:
    mixup=0 if e<20 else 0.2
    learn = get_learn(model=model, size=192, bs=32, mixup=mixup)
    learn.fit_fc(e, lr=4e-3, moms=(0.95,0.95), start_pct=0.72)
    res += [learn.recorder.metrics[-1][0].item()]
print([round(x, 6) for x in res], sum(res)/len(res))

data path   /root/.fastai/data/imagewoof2


/usr/local/lib/python3.6/dist-packages/torch/nn/functional.py:2854: UserWarning: The default behavior for interpolate/upsample with float scale_factor will change in 1.6.0 to align with other frameworks/libraries, and use scale_factor directly, instead of relying on the computed output size. If you wish to keep the old behavior, please set recompute_scale_factor=True. See the documentation of nn.Upsample for details. 
  warnings.warn("The default behavior for interpolate/upsample with float scale_factor will change "
/usr/local/lib/python3.6/dist-packages/torch/nn/functional.py:2854: UserWarning: The default behavior for interpolate/upsample with float scale_factor will change in 1.6.0 to align with other frameworks/libraries, and use scale_factor directly, instead of relying on the computed output size. If you wish to keep the old behavior, please set recompute_scale_factor=True. See the documentation of nn.Upsample for details. 
  warnings.warn("The default behavior for interpolate/u

Learn path /root/.fastai/data/imagewoof2


epoch,train_loss,valid_loss,accuracy,top_k_accuracy,time
0,1.846981,1.639446,0.495800,0.905319,02:37
1,1.591093,1.536333,0.552812,0.930262,02:35
2,1.458862,1.327474,0.651056,0.953678,02:35
3,1.291179,1.194189,0.718249,0.965895,02:35
4,1.076840,1.038393,0.795113,0.978875,02:35


/usr/local/lib/python3.6/dist-packages/torch/nn/functional.py:2854: UserWarning: The default behavior for interpolate/upsample with float scale_factor will change in 1.6.0 to align with other frameworks/libraries, and use scale_factor directly, instead of relying on the computed output size. If you wish to keep the old behavior, please set recompute_scale_factor=True. See the documentation of nn.Upsample for details. 
  warnings.warn("The default behavior for interpolate/upsample with float scale_factor will change "


...

/usr/local/lib/python3.6/dist-packages/torch/nn/functional.py:2854: UserWarning: The default behavior for interpolate/upsample with float scale_factor will change in 1.6.0 to align with other frameworks/libraries, and use scale_factor directly, instead of relying on the computed output size. If you wish to keep the old behavior, please set recompute_scale_factor=True. See the documentation of nn.Upsample for details. 
  warnings.warn("The default behavior for interpolate/upsample with float scale_factor will change "
/usr/local/lib/python3.6/dist-packages/torch/nn/functional.py:2854: UserWarning: The default behavior for interpolate/upsample with float scale_factor will change in 1.6.0 to align with other frameworks/libraries, and use scale_factor directly, instead of relying on the computed output size. If you wish to keep the old behavior, please set recompute_scale_factor=True. See the documentation of nn.Upsample for details. 
  warnings.warn("The default behavior for interpolate/u

Learn path /root/.fastai/data/imagewoof2


epoch,train_loss,valid_loss,accuracy,top_k_accuracy,time
0,1.899235,1.833637,0.422245,0.863833,02:35
1,1.651439,1.500513,0.561466,0.928481,02:35
2,1.477478,1.400202,0.616951,0.937898,02:35
3,1.328281,1.196329,0.703996,0.966149,02:35
4,1.108499,1.041227,0.792314,0.978112,02:34


/usr/local/lib/python3.6/dist-packages/torch/nn/functional.py:2854: UserWarning: The default behavior for interpolate/upsample with float scale_factor will change in 1.6.0 to align with other frameworks/libraries, and use scale_factor directly, instead of relying on the computed output size. If you wish to keep the old behavior, please set recompute_scale_factor=True. See the documentation of nn.Upsample for details. 
  warnings.warn("The default behavior for interpolate/upsample with float scale_factor will change "


torch.Size([1, 128, 48, 48])
torch.Size([1, 128, 48, 48])
torch.Size([1, 128, 48, 48])
torch.Size([1, 256, 24, 24])
torch.Size([1, 256, 24, 24])
torch.Size([1, 256, 24, 24])
torch.Size([1, 256, 24, 24])
torch.Size([1, 256, 24, 24])
torch.Size([1, 256, 24, 24])
torch.Size([1, 512, 12, 12])
torch.Size([1, 512, 12, 12])
torch.Size([1, 512, 12, 12])
torch.Size([1, 512, 12, 12])
torch.Size([1, 512, 12, 12])
torch.Size([1, 512, 12, 12])
torch.Size([1, 512, 12, 12])
torch.Size([1, 512, 12, 12])
torch.Size([1, 1024, 6, 6])
torch.Size([1, 1024, 6, 6])
torch.Size([1, 1024, 6, 6])
[0.795113, 0.78646, 0.792314] 0.7912955085436503


In [47]:
res

[0.8017306923866272, 0.7946042418479919, 0.8063120245933533]

In [13]:
epochs = [20,80]
for e in epochs:
    mixup=0 if e<20 else 0.2
    learn = get_learn(model=model, size=192, bs=32, mixup=mixup)
    learn.fit_fc(e, lr=4e-3, moms=(0.95,0.95), start_pct=0.72)
    res += [learn.recorder.metrics[-1][0].item()]
print([round(x, 6) for x in res], sum(res)/len(res))

data path   /root/.fastai/data/imagewoof2


/usr/local/lib/python3.6/dist-packages/torch/nn/functional.py:2854: UserWarning: The default behavior for interpolate/upsample with float scale_factor will change in 1.6.0 to align with other frameworks/libraries, and use scale_factor directly, instead of relying on the computed output size. If you wish to keep the old behavior, please set recompute_scale_factor=True. See the documentation of nn.Upsample for details. 
  warnings.warn("The default behavior for interpolate/upsample with float scale_factor will change "
/usr/local/lib/python3.6/dist-packages/torch/nn/functional.py:2854: UserWarning: The default behavior for interpolate/upsample with float scale_factor will change in 1.6.0 to align with other frameworks/libraries, and use scale_factor directly, instead of relying on the computed output size. If you wish to keep the old behavior, please set recompute_scale_factor=True. See the documentation of nn.Upsample for details. 
  warnings.warn("The default behavior for interpolate/u

Learn path /root/.fastai/data/imagewoof2


epoch,train_loss,valid_loss,accuracy,top_k_accuracy,time
0,1.990599,1.736880,0.443370,0.878341,02:35
1,1.815191,1.603337,0.513362,0.916518,02:35
2,1.689175,1.384134,0.622296,0.950115,02:35
3,1.585719,1.262384,0.686689,0.961059,02:35
4,1.502034,1.231501,0.699924,0.961822,02:35
5,1.433251,1.166047,0.723848,0.961313,02:35
6,1.382169,1.107457,0.752354,0.969458,02:35
7,1.323471,1.069485,0.770934,0.968694,02:35
8,1.303199,1.022692,0.793332,0.975821,02:35
9,1.257149,0.977906,0.811657,0.979130,02:35


torch.Size([1, 128, 48, 48])
torch.Size([1, 128, 48, 48])
torch.Size([1, 128, 48, 48])
torch.Size([1, 256, 24, 24])
torch.Size([1, 256, 24, 24])
torch.Size([1, 256, 24, 24])
torch.Size([1, 256, 24, 24])
torch.Size([1, 256, 24, 24])
torch.Size([1, 256, 24, 24])
torch.Size([1, 512, 12, 12])
torch.Size([1, 512, 12, 12])
torch.Size([1, 512, 12, 12])
torch.Size([1, 512, 12, 12])
torch.Size([1, 512, 12, 12])
torch.Size([1, 512, 12, 12])
torch.Size([1, 512, 12, 12])
torch.Size([1, 512, 12, 12])
torch.Size([1, 1024, 6, 6])
torch.Size([1, 1024, 6, 6])
torch.Size([1, 1024, 6, 6])
data path   /root/.fastai/data/imagewoof2


/usr/local/lib/python3.6/dist-packages/torch/nn/functional.py:2854: UserWarning: The default behavior for interpolate/upsample with float scale_factor will change in 1.6.0 to align with other frameworks/libraries, and use scale_factor directly, instead of relying on the computed output size. If you wish to keep the old behavior, please set recompute_scale_factor=True. See the documentation of nn.Upsample for details. 
  warnings.warn("The default behavior for interpolate/upsample with float scale_factor will change "
/usr/local/lib/python3.6/dist-packages/torch/nn/functional.py:2854: UserWarning: The default behavior for interpolate/upsample with float scale_factor will change in 1.6.0 to align with other frameworks/libraries, and use scale_factor directly, instead of relying on the computed output size. If you wish to keep the old behavior, please set recompute_scale_factor=True. See the documentation of nn.Upsample for details. 
  warnings.warn("The default behavior for interpolate/u

Learn path /root/.fastai/data/imagewoof2


epoch,train_loss,valid_loss,accuracy,top_k_accuracy,time
0,2.042191,1.820045,0.389921,0.874523,02:35
1,1.858240,1.689590,0.463222,0.906083,02:36
2,1.681571,1.434593,0.604225,0.943243,02:35
3,1.595757,1.319408,0.652329,0.952405,02:35
4,1.489135,1.232903,0.708323,0.961059,02:36
5,1.418969,1.155650,0.730466,0.966404,02:35
6,1.379181,1.067593,0.772716,0.975566,02:35
7,1.354473,1.068396,0.773988,0.972003,02:35
8,1.297986,1.025799,0.794095,0.972512,02:35
9,1.261426,0.986325,0.808857,0.978366,02:35


/usr/local/lib/python3.6/dist-packages/torch/nn/functional.py:2854: UserWarning: The default behavior for interpolate/upsample with float scale_factor will change in 1.6.0 to align with other frameworks/libraries, and use scale_factor directly, instead of relying on the computed output size. If you wish to keep the old behavior, please set recompute_scale_factor=True. See the documentation of nn.Upsample for details. 
  warnings.warn("The default behavior for interpolate/upsample with float scale_factor will change "


torch.Size([1, 128, 48, 48])
torch.Size([1, 128, 48, 48])
torch.Size([1, 128, 48, 48])
torch.Size([1, 256, 24, 24])
torch.Size([1, 256, 24, 24])
torch.Size([1, 256, 24, 24])
torch.Size([1, 256, 24, 24])
torch.Size([1, 256, 24, 24])
torch.Size([1, 256, 24, 24])
torch.Size([1, 512, 12, 12])
torch.Size([1, 512, 12, 12])
torch.Size([1, 512, 12, 12])
torch.Size([1, 512, 12, 12])
torch.Size([1, 512, 12, 12])
torch.Size([1, 512, 12, 12])
torch.Size([1, 512, 12, 12])
torch.Size([1, 512, 12, 12])
torch.Size([1, 1024, 6, 6])
torch.Size([1, 1024, 6, 6])
torch.Size([1, 1024, 6, 6])
[0.795113, 0.78646, 0.792314, 0.885722, 0.899211] 0.8317638158798217


In [1]:
res

NameError: ignored

conv: 1-to-1, conv1x1: 2-to-2

in 5, 5, 5, 20, and 80 epochs:
`0.804021, 0.805039, 0.811657, 0.884449, 0.893357`
(2.42 min per epoch)

`0.79944, 0.806058, 0.816238, 0.879359, 0.891066`

`0.796895, 0.789768, 0.787223, 0.878595, 0.893612`

with random shuffle
`0.807585, 0.803767, 0.819547, 0.87605, 0.889539`

`0.807076, 0.809621, 0.808094, 0.877577, 0.886994`

`[0.795113, 0.78646, 0.792314, 0.885722, 0.899211]`

In [13]:
for i in range(80):
    print('epoch {} {}'.format(i, learn.recorder.metrics[i][0].item()))

epoch 0 0.4275897145271301
epoch 1 0.5441588163375854
epoch 2 0.6617459654808044
epoch 3 0.6976329684257507
epoch 4 0.738355815410614
epoch 5 0.7701705098152161
epoch 6 0.7719521522521973
epoch 7 0.8063120245933533
epoch 8 0.8111478686332703
epoch 9 0.8045304417610168
epoch 10 0.81674724817276
epoch 11 0.8205650448799133
epoch 12 0.8297276496887207
epoch 13 0.8266734480857849
epoch 14 0.8322728276252747
epoch 15 0.8363451361656189
epoch 16 0.8355816006660461
epoch 17 0.8429625630378723
epoch 18 0.836090624332428
epoch 19 0.8493255376815796
epoch 20 0.8505980968475342
epoch 21 0.8536523580551147
epoch 22 0.828964114189148
epoch 23 0.8528887629508972
epoch 24 0.8561974763870239
epoch 25 0.8427080512046814
epoch 26 0.8505980968475342
epoch 27 0.8480529189109802
epoch 28 0.8567065596580505
epoch 29 0.8600152730941772
epoch 30 0.8556884527206421
epoch 31 0.8531432747840881
epoch 32 0.8488165140151978
epoch 33 0.8579791188240051
epoch 34 0.8475438952445984
epoch 35 0.855942964553833
epoch 36

In [ ]:
for i in range(80):
    print('epoch {} {}'.format(i, learn.recorder.metrics[i][0].item()))

epoch 0 0.41995418071746826
epoch 1 0.5189615488052368
epoch 2 0.6151692271232605
epoch 3 0.667345404624939
epoch 4 0.6917790770530701
epoch 5 0.736065149307251
epoch 6 0.7696614861488342
epoch 7 0.7630440592765808
epoch 8 0.7900229096412659
epoch 9 0.7946042418479919
epoch 10 0.7956222891807556
epoch 11 0.8086026906967163
epoch 12 0.8170017600059509
epoch 13 0.8162382245063782
epoch 14 0.8340544700622559
epoch 15 0.8376176953315735
epoch 16 0.8429625630378723
epoch 17 0.8421990275382996
epoch 18 0.8462713360786438
epoch 19 0.8470348715782166
epoch 20 0.8449987173080444
epoch 21 0.8488165140151978
epoch 22 0.8574700951576233
epoch 23 0.8546704053878784
epoch 24 0.8368541598320007
epoch 25 0.8640875816345215
epoch 26 0.8523797392845154
epoch 27 0.8539068698883057
epoch 28 0.8495800495147705
epoch 29 0.855942964553833
epoch 30 0.8521252274513245
epoch 31 0.8572155833244324
epoch 32 0.8597607612609863
epoch 33 0.8610333204269409
epoch 34 0.8546704053878784
epoch 35 0.8549249172210693
epoc

In [ ]:
for i in range(80):
    print('epoch {} {}'.format(i, learn.recorder.metrics[i][0].item()))

epoch 0 0.39373886585235596
epoch 1 0.5291422605514526
epoch 2 0.6054975986480713
epoch 3 0.6536014080047607
epoch 4 0.6759989857673645
epoch 5 0.7179943919181824
epoch 6 0.7459913492202759
epoch 7 0.7755153775215149
epoch 8 0.7806057333946228
epoch 9 0.8131840229034424
epoch 10 0.814202070236206
epoch 11 0.8157292008399963
epoch 12 0.8022397756576538
epoch 13 0.8228557109832764
epoch 14 0.8256554007530212
epoch 15 0.8416900038719177
epoch 16 0.8315092921257019
epoch 17 0.8355816006660461
epoch 18 0.8437261581420898
epoch 19 0.8378722071647644
epoch 20 0.8460168242454529
epoch 21 0.8393993377685547
epoch 22 0.8513616919517517
epoch 23 0.8485620021820068
epoch 24 0.8480529189109802
epoch 25 0.8567065596580505
epoch 26 0.8511071801185608
epoch 27 0.8556884527206421
epoch 28 0.8498345613479614
epoch 29 0.853397786617279
epoch 30 0.8612878322601318
epoch 31 0.8569610714912415
epoch 32 0.8605242967605591
epoch 33 0.8615424036979675
epoch 34 0.8518707156181335
epoch 35 0.858488142490387
epoc